In [ ]:
import numpy as np
import pandas as pd
import scipy.io as sio
import mat73,os,json

%cd /home/yuqiyou/faceprf_dataprocesscode/faceprf_vol

subjs = ['CN040','CN041','CN042','CN043','CN044','CN045','CN055','CN056']
hemi = ['lh','rh']
tasks = ['face','fixation']
roi_labels = ['V1','V2','V3','hV4','IOG','pFus','mFus']

nsubj = len(subjs)
nvoxel = 100
ntrial = 1280
ntrialperpos = 80




### load data

faceprfres_all = dict()
for subj_i in range(len(subjs)):
    os.chdir('/home/data/rawdata/facePRF/'+subjs[subj_i]+'_faceprf/faceprfanalyze_vol')
    # 0-vind, 1-R2, 2-x, 3-y, 4-ecc, 5-ang, 6-size, 7-expt, 8-gain
    faceprfres_all[subjs[subj_i]] = mat73.loadmat(subjs[subj_i]+'_faceprfresults.mat',only_include = ['faceprfres'])['faceprfres']
    
glmdres_all = dict()
stimposicond_all = np.ones([len(tasks),len(subjs),ntrial])*np.nan
for subj_i in range(len(subjs)):
    os.chdir('/home/data/rawdata/facePRF/'+subjs[subj_i]+'_faceprf/faceprfanalyze_vol')
    glmdres_all[subjs[subj_i]] = mat73.loadmat(subjs[subj_i]+'_PRFdatasets.mat',only_include = ['data'])['data']
    stimposicond = mat73.loadmat(subjs[subj_i]+'_PRFdatasets.mat',only_include = ['stimposicond'])['stimposicond']
    stimposicond_all[0,subj_i,:] = stimposicond['face']
    stimposicond_all[1,subj_i,:] = stimposicond['fixation']   




## voxel selection: GLM R2 & pRF R2

nvoxel_glm = 200

v_sel_all = np.ones([len(roi_labels),len(subjs),nvoxel],dtype=int)*(-1)
r2_sel_all = np.ones([len(roi_labels),len(subjs),nvoxel,2])*np.nan
beta_sel_all = np.ones([len(tasks),len(roi_labels),len(subjs),nvoxel,ntrial])*np.nan
prf_sel_all = np.ones([len(tasks),len(roi_labels),len(subjs),nvoxel,8])*np.nan

for subj_i in range(len(subjs)):

    os.chdir('/home/data/rawdata/facePRF/'+subjs[subj_i]+'_faceprf/faceprfanalyze_vol')
    glmres = glmdres_all[subjs[subj_i]]
    faceprfres = faceprfres_all[subjs[subj_i]]
    beta = dict()
    for task_i in range(len(tasks)):
        beta[tasks[task_i]] = glmres['beta'][tasks[task_i]]
    prf = dict()
    for task_i in range(len(tasks)):
        prf[tasks[task_i]] = faceprfres[tasks[task_i]]

    # glm & prf r2
    vind = (prf['fixation'][:,0]-1).astype('int')
    r2 = pd.DataFrame(index=vind,columns=['glm_r2','prf_r2'])
    r2['glm_r2'] = glmres['R2'].mean(axis=1)[vind]
    r2['prf_r2'] = prf['fixation'][:,1]

    #roi_all = np.load(subjs[subj_i]+'_ROI.npy',allow_pickle=True).item() # from 0
    with open(subjs[subj_i]+'_ROI.json', 'r', encoding='utf-8') as file:
        roi_all = json.load(file)
    for roi_i in range(len(roi_labels)):

        # roi
        vroi = np.append(roi_all['lh'][roi_labels[roi_i]],roi_all['rh'][roi_labels[roi_i]]) # whole brain index
        r2_roi = r2.loc[vroi]
    
        # voxels with unique responses
        beta_value = glmres['beta']['fixation']
        beta_roi = beta_value[r2_roi.index,:]
        beta_unique,unique_ind,repeat_count = np.unique(beta_roi,return_index=True,return_counts=True,axis=0) # unique_ind: array_ind
        r2_roi_unique = r2_roi.iloc[unique_ind,:]

        # glm r2 top 200
        r2_roi_unique_glmsort = r2_roi_unique.sort_values(by='glm_r2',ascending=False,inplace=False).iloc[:nvoxel_glm,:]


        # prf r2 top 100
        r2_roi_unique_glmsort_prfsort = r2_roi_unique_glmsort.sort_values(by='prf_r2',ascending=False,inplace=False).iloc[:nvoxel,:]

        # voxel & data
        v_sel_all[roi_i,subj_i,:] = r2_roi_unique_glmsort_prfsort.index
        r2_sel_all[roi_i,subj_i,:,:] = np.array(r2_roi_unique_glmsort_prfsort)
        beta_sel_all[0,roi_i,subj_i,:,:] = beta[tasks[0]][v_sel_all[roi_i,subj_i,:],:]
        beta_sel_all[1,roi_i,subj_i,:,:] = beta[tasks[1]][v_sel_all[roi_i,subj_i,:],:]
        ind = [np.where(vind==v)[0] for v in v_sel_all[roi_i,subj_i,:]]
        prf_sel_all[0,roi_i,subj_i,:,:] = prf[tasks[0]][ind,1:].reshape(nvoxel,8)
        prf_sel_all[1,roi_i,subj_i,:,:] = prf[tasks[1]][ind,1:].reshape(nvoxel,8)
        
%cd /home/yuqiyou/faceprf_dataprocesscode/faceprf_vol
np.savez('data/selectdata-both',v_sel_all=v_sel_all,r2_sel_all=r2_sel_all,
         stimposicond_all=stimposicond_all,
         beta_sel_all=beta_sel_all,prf_sel_all=prf_sel_all)             

In [ ]:
for subj_i in range(0,len(subjs)):

    print('glm r2 range: {:.2f}% ~ {:.2f}%'.format(np.min(r2_sel_all[:,subj_i,:,0])*100,np.max(r2_sel_all[:,subj_i,:,0])*100),end='\t')
    print('prf r2 range: {:.2f}% ~ {:.2f}%'.format(np.min(r2_sel_all[:,subj_i,:,1]),np.max(r2_sel_all[:,subj_i,:,1])))

glm r2 range: 43.46% ~ 82.58%	prf r2 range: 28.37% ~ 75.95%
glm r2 range: 39.36% ~ 81.51%	prf r2 range: 34.17% ~ 77.95%
glm r2 range: 39.87% ~ 85.50%	prf r2 range: 34.38% ~ 78.08%
glm r2 range: 45.87% ~ 79.77%	prf r2 range: 25.71% ~ 59.11%
glm r2 range: 30.49% ~ 83.19%	prf r2 range: 33.64% ~ 79.25%
glm r2 range: 40.22% ~ 80.23%	prf r2 range: 32.34% ~ 76.94%
glm r2 range: 32.69% ~ 81.51%	prf r2 range: 28.86% ~ 82.40%
glm r2 range: 44.44% ~ 81.84%	prf r2 range: 22.85% ~ 68.30%
